In [2]:
from utils import *
from Bio import Entrez
import requests
import pandas as pd
import json
from LLM_prompts import analysis_introduction,analysis_citation
import logging
import re,datetime



In [15]:
df = pd.read_csv('pmc_queue.csv')

TypeError: read_csv() got an unexpected keyword argument 'ignore_index'

In [9]:
df

,pubmed_id
0,35409106
1,35409106


In [10]:
df = pd.concat([df, pd.DataFrame([{'pubmed_id':35409106}])])

In [13]:
df=df[1:]

In [14]:
df

,pubmed_id
1,35409106
0,35409106


In [16]:
df['pubmed_id']

1    35409106
2    35409106
Name: pubmed_id, dtype: int64

In [2]:
log_file = "my_log_file.log"
logger = setup_logger(log_file)

ref = {'Citation': 'This is a sample citation.', 'ArticleIdList': '12345'}
cite = "Citation: {}"
logger.info(cite, ref['ArticleIdList'])
logger.info(ref['Citation'])

2023-12-04 14:10:47,083 - utils - INFO - Citation: 12345
2023-12-04 14:10:47,084 - utils - INFO - This is a sample citation.


In [5]:
logger.info("This is a message with two placeholders: {} and {}".format('1', '2'))

2023-12-04 13:55:26,343 - utils - INFO - This is a message with two placeholders: 1 and 2


In [52]:
def setup_logger(log_file):
    # 创建一个Logger对象
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)  # 设置日志级别为INFO或其他适当级别

    # 创建一个文件处理程序，用于将日志写入指定的文件
    file_handler = logging.FileHandler(log_file)

    # 创建一个格式化程序，用于指定日志的格式
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)

    # 将文件处理程序添加到Logger对象
    logger.addHandler(file_handler)

    return logger

In [ ]:
log_file = "my_log_file.log"
logger = setup_logger(log_file)
logger.info("这是一个信息级别的日志消息")

In [2]:

df = pd.read_csv('pmc_queue.csv',header=None)
print(df)

your_email = "shou@nd.edu"  # 请用您的邮箱地址替换
download_folder = "pdfs"
img_folder = "imgs"
Entrez.email = your_email  # 请将其替换为您自己的电子邮件地址
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"
headers = {"User-Agent": user_agent}

success_download_flag = False
pubmed_id = str(df.iloc[0][0])
article = Entrez.efetch(db="pubmed", id=pubmed_id, retmode="xml")
xml_data = Entrez.read(article)
abstract = xml_data['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText']
title = xml_data['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
ref = xml_data["PubmedArticle"][0]["PubmedData"]["ReferenceList"][0]["Reference"]

article.close()


formatted_data = json.dumps(xml_data, indent=4)
with open('text.xml','w') as f:
    f.write(formatted_data)

try:
    # 尝试获取PMC ID，以构造PDF URL
    handle = Entrez.elink(dbfrom="pubmed", id=pubmed_id, linkname="pubmed_pmc")
    record = Entrez.read(handle)
    handle.close()

    pmc_id_links = record[0]["LinkSetDb"]
    if pmc_id_links and 'Link' in pmc_id_links[0]:
        pmc_id = pmc_id_links[0]["Link"][0]["Id"]
        pdf_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/PMC{pmc_id}/pdf/"
        file_path = os.path.join(download_folder, f"PMC{pmc_id}.pdf")

        download_pdf_with_retry(pdf_url, file_path, headers=headers, num_retries=3)
        success_download_flag =True
    else:
        print(f"No PMC article available for PubMed ID: {pubmed_id}")
        success_download_flag = False

except Exception as e:
    print(f"Error occurred for PubMed ID {pubmed_id}: {e}")
    success_download_flag = False


          0
0  35409106


In [42]:
ref[0]['Citation']

'Han Y., Li X., Zhang Y., Han Y., Chang F., Ding J. Mesenchymal Stem Cells for Regenerative Medicine. Cells. 2019;8:886. doi: 10.3390/cells8080886.'

In [22]:
for r in ref:
    print(len(r['ArticleIdList']),r['ArticleIdList'])
    for t in r['ArticleIdList']:
        if t.attributes['IdType'] == 'pmc':
            print(t)



3 [StringElement('10.3390/cells8080886', attributes={'IdType': 'doi'}), StringElement('PMC6721852', attributes={'IdType': 'pmc'}), StringElement('31412678', attributes={'IdType': 'pubmed'})]
PMC6721852
3 [StringElement('10.15283/ijsc18034', attributes={'IdType': 'doi'}), StringElement('PMC6657936', attributes={'IdType': 'pmc'}), StringElement('31022994', attributes={'IdType': 'pubmed'})]
PMC6657936
3 [StringElement('10.1016/j.stem.2018.05.004', attributes={'IdType': 'doi'}), StringElement('PMC6434696', attributes={'IdType': 'pmc'}), StringElement('29859173', attributes={'IdType': 'pubmed'})]
PMC6434696
3 [StringElement('10.1111/j.1582-4934.2011.01258.x', attributes={'IdType': 'doi'}), StringElement('PMC4373326', attributes={'IdType': 'pmc'}), StringElement('21251211', attributes={'IdType': 'pubmed'})]
PMC4373326
3 [StringElement('10.3390/cells10092371', attributes={'IdType': 'doi'}), StringElement('PMC8472468', attributes={'IdType': 'pmc'}), StringElement('34572020', attributes={'IdTyp

In [30]:
if success_download_flag:
    # try:
    introduction = extract_introduction_from_pdf(file_path)
    img_path = os.path.join(img_folder, f"PMC{pmc_id}")
    print(img_path)
    legends,imgs = extract_images_and_text_below(file_path,img_path)
    pattern = r'\[\d+([–-]\d+)?(,\d+([–-]\d+)?)*\]'
    matches = re.finditer(pattern, introduction)
    start = 0

    for m in matches:
        print("==========================================")
        citation_nums = m.group()
        citation = introduction[start:m.end()]
        start = m.end()+1
        print(citation_nums,'\n',citation)
        
        result = analysis_citation(citation)
        print("-------------------------------------------")
        print(result)
        cites = []
        if "Conclusion: Yes" in result:
            if '–' in citation_nums:
                s = citation_nums[1:-1].split('–')[0]
                e = citation_nums[1:-1].split('–')[1]
                for i in range(int(s),int(e)+1):
                    cites.append(i)

            elif '-' in citation_nums:
                s = citation_nums[1:-1].split('-')[0]
                e = citation_nums[1:-1].split('-')[1]
                for i in range(int(s),int(e)+1):
                    cites.append(i)

            elif ',' in citation_nums:

                for i in citation_nums[1:-1].split(','):
                    cites.append(int(i))
            else:

                cites.append(int(citation_nums[1:-1]))
            print("cite nums:",cites)            
            
            

imgs\PMC8998189
4 figures and legends extracted.
[1–3] 
 Introduction
Mesenchymal stem cells (MSCs) have been consistently reported as promising candi-
dates in functional tissue engineering and regenerative therapies [1–3]
-------------------------------------------
Analysis process: 

Based on the provided paragraph, it suggests that the article is discussing the potential of mesenchymal stem cells (MSCs) in functional tissue engineering and regenerative therapies. There is no specific mention of the growth of these cells in different oxygen concentration environments. 

Conclusion: No.
[4] 
  The umbilical cord
comprises a speciﬁc stem cell niche where precise oxygen (O2) levels are paramount to
maintain and optimize stem cell function. In this tissue, umbilical cord-mesenchymal stem
cells (UC-MSCs) reside under hypoxic conditions, ranging from less than 1% O2 (severe
hypoxia) to 5% O2 (moderate hypoxia) [4]
-------------------------------------------
Analysis process: The article d

In [25]:
citation_nums

'[20]'

In [28]:
if '–' in citation_nums:
    s = citation_nums[1:-1].split('–')[0]
    e = citation_nums[1:-1].split('–')[1]
    print(s,e)

elif '-' in citation_nums:
    s = citation_nums[1:-1].split('-')[0]
    e = citation_nums[1:-1].split('-')[1]
    print(s,e)

elif ',' in citation_nums:
    print(citation_nums[1:-2].split(','))
else:
    print(citation_nums[1:-1])

20


In [49]:
list(df.iloc[:,0])

[1, 1]

In [35]:
df.loc[len(df)] = 1

In [48]:
type([])

list

In [37]:
df = df.drop(df.index[0])

In [40]:
df.to_csv('test.csv',header=None,index=None)